In [1]:
import pandas as pd
import numpy as np
import json
import datetime
import glob

# 1. Load transaction data

In [2]:
# Load all data and concat
csvs = glob.glob('/gh/data/personal-data-requests/BoA/*.csv')
dfs = []
for csv in csvs:
    df_temp = pd.read_csv(csv)
    dfs.append(df_temp)
df_web = pd.concat(dfs).drop_duplicates().dropna(how='all', axis=0)

# Process cols
cols_keep = ['date', 'Amount', 'Original Description', 'Category', 'Account Name', 'Simple Description']
df_web['date']  = pd.to_datetime(df_web['Date'])
df_web['Amount'] = np.array([x.replace(',','') for x in df_web['Amount'].astype(str)], dtype=float)
df_web = df_web[cols_keep]
df_web.head()

/Users/scott/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  import sys


,date,Amount,Original Description,Category,Account Name,Simple Description
0,2019-08-04,135.32,PENDING PAYMENT,Credit Card Payments,Bank of America - Credit Card - Bank of Americ...,PENDING PAYMENT
1,2019-08-04,-77.52,TURO INC,Other Expenses,Bank of America - Credit Card - Bank of Americ...,TURO INC
2,2019-08-03,-39.61,JAPANESE HOUSE SAN FRANCISCOCA,Restaurants/Dining,Bank of America - Credit Card - Bank of Americ...,JAPANESE HOUSE SAN FRANCISCOCA
3,2019-08-03,-5.05,SQ *STEEP CREAMERY SAN FRANCISCOCA,Restaurants/Dining,Bank of America - Credit Card - Bank of Americ...,The Creamery
4,2019-08-01,-3.05,UBER EATS HELP.UBER.COMCA,Restaurants/Dining,Bank of America - Credit Card - Bank of Americ...,Uber Eats


### 1b. Isolate deposits / transfers

In [5]:
df_money = df_web[df_web['Amount'].astype(float) > 0]
df_money.loc[[1472]]

,date,Amount,Original Description,Category,Account Name,Simple Description
1472,2017-07-11,1000.0,Online payment from CHK 0,Credit Card Payments,Bank of America - Credit Card - Bank of Americ...,Online payment from CHK 0


# 2. Import pdf data from table
* Tabula - Does not really work. Gets some tables but many are missing
* Camelot - does not read any tables
* Used Excalibur gui

In [6]:
pdf_csvs = glob.glob('/gh/data/personal-data-requests/BoA/excalibur/*/*.csv')
dfs = []
for csv in pdf_csvs:
    # Read csv
    df_temp = pd.read_csv(csv)
    category = None
    
    # If the first row is the category
    if 'Unnamed: 2' in df_temp.columns:
        category = df_temp.columns[0]
        df_temp = pd.read_csv(csv, skiprows=1)
        
    # If there are no columns
    if 'Location' not in df_temp.columns:
        df_temp = pd.read_csv(csv, names=['Date\nDescription', 'Location', 'Amount'])
        
        
    # If Date and Description are stuck together
    if 'Date\nDescription' in df_temp.columns:
        df_temp['date'] = pd.to_datetime([x[:8] for x in df_temp['Date\nDescription']])
        df_temp['description'] = [x[8:] for x in df_temp['Date\nDescription']]
        df_temp = df_temp.drop('Date\nDescription', axis=1)
        
    # Remove 'deduct' column
    if 'Deduct' in df_temp.columns:
        df_temp.drop('Deduct', axis=1, inplace=True)
  
    # Add df
    df_temp['pdf_category'] = category
    dfs.append(df_temp)
    
# Concat and process data
df_pdf = pd.concat(dfs).reset_index(drop=True)
df_pdf = df_pdf.rename(columns={'Location': 'city', 'Amount': 'amount'})

# Remove 'CR' (negative) credits for simplicity. it's not too many columns
df_CR = df_pdf[df_pdf['amount'].astype(str).str.contains('CR')]
df_pdf = df_pdf[~df_pdf['amount'].astype(str).str.contains('CR')]
df_pdf['amount'] = np.array([x.replace(',','') for x in df_pdf['amount'].astype(str)], dtype=float)

df_pdf.head()

,city,amount,date,description,pdf_category
0,"NEW YORK,NY",32.00,2016-12-27,MTA MVMN116-NOSTRAND A,Other Travel/Transportation
1,"SAN DIEGO,CA",5.99,2016-08-22,SPROUTS FARMERS MAR,None
2,"SAN MARCOS,CA",13.50,2016-08-23,SENOR PANCHOS - SAN MA,None
3,"OCEANSIDE,CA",10.24,2016-10-24,I LOVE BAGELS - OCEANS,None
4,"WWW.YELP.COM,CA",23.19,2016-08-24,YELPINCEAT24 PUNJABI,Services


# 3. Merge pdf and web data

In [7]:
df_web_merge = df_web[df_web['Amount'] < 0].rename(columns={'Amount': 'amount'})
df_web_merge['amount'] = -df_web_merge['amount']
df_web_merge = df_web_merge.drop_duplicates(subset=['date', 'amount'])

df_both = df_pdf.merge(df_web_merge, on=['date', 'amount'], how='left')
df_both.to_csv('/gh/data/personal-data-requests/BoA/web_pdf_merge.csv', index_label=None)
df_both.head()

,city,amount,date,description,pdf_category,Original Description,Category,Account Name,Simple Description
0,"NEW YORK,NY",32.00,2016-12-27,MTA MVMN116-NOSTRAND A,Other Travel/Transportation,NaN,NaN,NaN,NaN
1,"SAN DIEGO,CA",5.99,2016-08-22,SPROUTS FARMERS MAR,None,NaN,NaN,NaN,NaN
2,"SAN MARCOS,CA",13.50,2016-08-23,SENOR PANCHOS - SAN MA,None,NaN,NaN,NaN,NaN
3,"OCEANSIDE,CA",10.24,2016-10-24,I LOVE BAGELS - OCEANS,None,NaN,NaN,NaN,NaN
4,"WWW.YELP.COM,CA",23.19,2016-08-24,YELPINCEAT24 PUNJABI,Services,NaN,NaN,NaN,NaN
